In [55]:
import xml.etree.ElementTree as et
import pandas as pd
from sox import Transformer
from os import path, walk

# disable truncated columns
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

# use full siza of display / can be enabled on bigger screens
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [58]:
def parse_sentence(sentence_node):
    sentence = ""
    start = 99999999999999
    end = 0
    
    for t in sentence_node.iterfind("t"):
        sentence += t.text + " "
        for n in t.iterfind("n"):
            if n.get("start") != None and start > int(n.get("start")):
                start = int(n.get("start"))
            if n.get("end") != None and end < int(n.get("end")):
                end = int(n.get("end"))

    return {"sentence": [sentence], "start": [start], "end": [end], "length": [end - start]}

def parse_article_header_description(xroot):
    columns = ["sentence", "start", "end", "length"]
    df = pd.DataFrame(columns=columns)
    
    for d in xroot.iterfind("d"):
        for p in d.iterfind("p"):
            for s in p.iterfind("s"):
                sentence_dict = parse_sentence(s)
                df = pd.concat([df, pd.DataFrame(sentence_dict)])
                
    return df
                
def parse_sections(xroot):
    columns = ["sentence", "start", "end", "length"]
    df = pd.DataFrame(columns=columns)
    
    for d in xroot.iterfind("d"):
        for section in d.iterfind("section"):
            for sectioncontent in section.iterfind("sectioncontent"):
                for p in sectioncontent.iterfind("p"):
                    for s in p.iterfind("s"):
                        sentence_dict = parse_sentence(s)
                        df = pd.concat([df, pd.DataFrame(sentence_dict)])
                            
    return df

def _convert_wav(path_prefix, filepath, sentence):
    _maybe_convert_wav(mp3_path, wav_path)
    frames = int(subprocess.check_output(['soxi', '-s', wav_path], stderr=subprocess.STDOUT))
    wav_file_size = path.getsize(wav_path)

    status = "no_status"

    if int(frames/SAMPLE_RATE*1000/10/2) < len(str(sentence)):
        # Excluding samples that are too short to fit the transcript
        status = "too_short"
    elif frames/SAMPLE_RATE > MAX_SECS:
        # Excluding very long samples to keep a reasonable batch-size
        status = "too_long"
    else:
        status = "ok"

    return pd.Series([wav_path, wav_file_size, status])

In [59]:
def parse_file(base_folder):
    filepath = path.join(base_folder, "aligned.swc")
    
    xtree = et.parse(filepath)
    xroot = xtree.getroot()
    df_header = parse_article_header_description(xroot)
    df_sections = parse_sections(xroot)
    df = pd.concat([df_header, df_sections])
    df["base_folder"] = base_folder
    df["status"] = "no_status"
    
    return df

def create_worklist(base_folder_path):
    df_complete_columns = ["sentence", "start", "end", "length", "base_folder", "status"]
    df_complete = pd.DataFrame(columns=df_complete_columns)
    
    for root, dirnames, filenames in walk(base_folder_path):
        for dirname in dirnames:
            file_path = path.join(base_folder_path, dirname)
            if path.exists(path.join(file_path, "aligned.swc")):
                df_complete = pd.concat([df_complete, parse_file(file_path)])
        break
        
    return df_complete.reset_index().drop("index", axis=1)

basepath = "../corpora/wiki/german/"

dff = create_worklist(basepath)
dff

,sentence,start,end,length,base_folder,status
0,"Die Resolution 1860 des UN-Sicherheitsrates ist eine völkerrechtlich bindende UN-Resolution , die der Sicherheitsrat der Vereinten Nationen am 8. Januar 2009 auf seiner 6063 .",18930,34180,15250,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
1,Sitzung als Reaktion auf den sogenannten Gaza-Krieg bei einer Enthaltung durch die Vereinigten Staaten einstimmig verabschiedet hat .,34320,43120,8800,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
2,In der Resolution wurden die Konfliktparteien Israel und die palästinensische radikal-islamistische Hamas zu einer sofortigen Waffenruhe aufgefordert .,43840,53170,9330,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
3,Außerdem wird Israel zum Rückzug seiner Streitkräfte aus dem Autonomiegebiet aufgefordert .,53590,59540,5950,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
4,"Die US-Außenministerin Condoleezza Rice begründete ihre Stimmenthaltung damit , Amerika wollte zunächst die ägyptischen Friedensvermittlungen abwarten , aber dennoch der Resolution nicht im Wege stehen , da der UN-Beschluss ein Schritt in die richtige Richtung sei .",60730,76970,16240,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
5,Politische Beobachter hatten mit einem Veto gerechnet .,77170,78710,1540,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
6,Die UN-Resolution 1860 enthält unter anderem folgende Punkte :,99999999999999,0,-99999999999999,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
7,"Der Sicherheitsrat fordert einen unverzüglichen , dauerhaften und vollständig eingehaltenen Waffenstillstand , der zum völligen Rückzug der israelischen Streitkräfte aus Gaza führt .",87560,97110,9550,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
8,"Er ruft alle UN-Mitgliedsstaaten auf , sich verstärkt für Vereinbarungen und Garantien in Gaza einzusetzen , um eine dauerhafte Waffenruhe zu erreichen , den Waffenschmuggel zu unterbinden und die Grenzübergänge auf Dauer wiederzueröffnen .",97610,111420,13810,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status
9,Dauerhafte und geregelte Öffnung der Grenzübergänge für Menschen und Güter .,111880,117320,5440,../corpora/wiki/german/Resolution_1860_des_UN-Sicherheitsrates,no_status


In [38]:
import hashlib

SAMPLE_RATE = 16000
for sen in s:
    m = hashlib.sha256()
    m.update(sen[0].encode())
    out_filename = path.join(basepath, m.hexdigest() + ".wav")
    in_filename = path.join(basepath, "audio.ogg")
    
    transformer = Transformer()
    transformer.trim(sen[1]/1000.0, sen[2]/1000.0)
    transformer.convert(samplerate=SAMPLE_RATE)
    transformer.build(in_filename, out_filename)
    
    print(out_filename.split("/")[-1], sen, sen[1]/1000.0, sen[2]/1000.0)

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [40]:
dff["sentence"]

0     Sitzung als Reaktion auf den sogenannten Gaza-Krieg bei einer Enthaltung durch die Vereinigten Staaten einstimmig verabschiedet hat                                                                                                                                                                                                                                            
1     Dauerhafte und geregelte Öffnung der Grenzübergänge für Menschen und Güter                                                                                                                                                                                                                                                                                                     
2     Der Sicherheitsrat begrüßt sowohl den ägyptischen Vorstoß einen Waffenstillstand zuwege zu bringen als auch andere derzeit laufende regionale und internationale Initiativen                                                                          